In [ ]:
!pip install pandas numpy matplotlib yfinance

In [ ]:
import time
import logging
# from kiteconnect import KiteConnect
import math
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os

# ---------------------------------------------
# CONFIGURATIONS & USER INPUTS
# ---------------------------------------------

In [ ]:
# Trading Mode: 'capital' or 'quantity'
TRADING_MODE = 'quantity'  # Change to 'capital' for capital-based trading

# Parameters for Capital-Based Trading
Y_CAPITAL = 100000.0          # Total capital dedicated to this strategy in INR

# Parameters for Quantity-Based Trading
QUANTITY = 1000                 # Fixed number of shares to trade

# Common Strategy Parameters
INSTRUMENT = "NSE:SBIN"
X_PROFIT = 2.0
RSI_PERIOD = 14
SUPER_TREND_PERIOD = 10
SUPER_TREND_MULTIPLIER = 3
TIMEFRAME = "5minute"
RSI_BUY_THRESHOLD = 30 #make a range here
RSI_SELL_THRESHOLD = 60
STOP_LOSS = 10.0
CHECK_INTERVAL = 0

# CSV with historical data for paper trading
CSV_FILE = "sbin_ns.csv"     # Replace with your actual historical data file

# Output report file
REPORT_FILE_TXT = "trade_report.txt"
REPORT_FILE = "trade_report.csv"

# ---------------------------------------------
# LOGGING CONFIG
# ---------------------------------------------

In [ ]:
# logging.basicConfig(level=logging.INFO, format='%(asctime)s [%(levelname)s]: %(message)s')
# Configure logging
logging.basicConfig(filename='app.log',
                    level=logging.DEBUG,
                    force=True, # Resets any previous configuration
                    )

logging.info("Logging is set up and ready!")

# ---------------------------------------------
# KITE CONNECT SETUP
# ---------------------------------------------

In [ ]:
kite = None
if TRADING_MODE == 'capital' or TRADING_MODE == 'quantity':
    # Assuming live trading requires Kite API credentials
    # For paper trading, these are not required
    pass

# For live trading, uncomment and set your API credentials
# API_KEY = "your_api_key_here"
# API_SECRET = "your_api_secret_here"
# ACCESS_TOKEN = "your_access_token_here"

# Uncomment the following block for live trading setup
# if TRADING_MODE == 'live':
#     if not API_KEY or not API_SECRET or not ACCESS_TOKEN:
#         logging.error("API_KEY, API_SECRET, ACCESS_TOKEN must be set for live trading!")
#         raise ValueError("Invalid API keys or tokens for live trading.")
#
#     kite = KiteConnect(api_key=API_KEY)
#     kite.set_access_token(ACCESS_TOKEN)
#     # Test the connection
#     try:
#         profile = kite.profile()
#         logging.info(f"Successfully authenticated with Zerodha. User: {profile['user_name']}")
#     except Exception as e:
#         logging.error(f"Failed to authenticate with Zerodha: {e}")
#         raise e

# ---------------------------------------------
# PAPER DATA HANDLER (For Paper Trading)
# ---------------------------------------------

In [ ]:
class PaperDataHandler:
    """
    Simulates fetching of historical and 'live' data from a local CSV for paper trading.
    - Loads a CSV with OHLC data.
    - Provides methods to get the latest candle slice for RSI calculation.
    - Provides a method to get the current LTP (simulated as the close of the latest candle).
    - Advances a pointer through the CSV rows to simulate time passing.
    """

    def __init__(self, csv_file):
        self.csv_file = csv_file
        self.df = None
        self.current_index = None
        self.load_data()

    def load_data(self):
      if not os.path.exists(self.csv_file):
          logging.error(f"CSV file {self.csv_file} not found. Exiting gracefully...")
          self.df = pd.DataFrame()
          return

      self.df = pd.read_csv(self.csv_file)

      # Convert column names to lowercase
      self.df.columns = map(str.lower, self.df.columns)

      # Validate data
      required_cols = {'date', 'open', 'high', 'low', 'close', 'volume'}
      if not required_cols.issubset(set(self.df.columns)):
          logging.error(f"CSV file {self.csv_file} does not have the required columns: {required_cols}")
          self.df = pd.DataFrame()
          return

      # Handle date parsing with error handling
      try:
          self.df['date'] = pd.to_datetime(self.df['date'], dayfirst=True, errors='coerce')
      except Exception as e:
          logging.error(f"Error parsing dates: {e}")
          self.df = pd.DataFrame()
          return

      # Drop rows where date parsing failed
      if self.df['date'].isnull().any():
          logging.warning(f"Some rows have invalid date formats and will be dropped.")
          self.df = self.df.dropna(subset=['date'])

      # Sort by date if not sorted
      self.df = self.df.sort_values('date').reset_index(drop=True)

      if self.df.empty:
          logging.error("CSV data is empty after processing. Exiting...")
          return

      # Start from the RSI_PERIOD-th candle to have enough data for RSI calculation
      self.current_index = RSI_PERIOD

    def get_historical_candles(self, lookback=100):
        """
        Returns a historical slice of candles up to current_index for RSI calculation.
        If current_index is None or data insufficient, return empty df.
        """
        if self.df.empty or self.current_index is None:
            return pd.DataFrame()

        start_idx = max(0, self.current_index - lookback)
        return self.df.iloc[start_idx:self.current_index]

    def get_current_ltp(self):
        """
        Returns the current candle's close price as LTP.
        If no more data is available, return None.
        """
        if self.df.empty or self.current_index is None:
            return None
        if self.current_index >= len(self.df):
            return None
        return self.df.iloc[self.current_index]['close']

    def advance_time(self):
        """
        Move to the next candle, simulating passing of time.
        If no more data, return False to indicate we are done.
        """
        if self.df.empty or self.current_index is None:
            return False
        self.current_index += 1
        if self.current_index >= len(self.df):
            # No more data available
            return False
        return True

# Initialize PaperDataHandler if PAPER_TRADING

In [ ]:
paper_data_handler = None
if TRADING_MODE in ['capital', 'quantity']:
    paper_data_handler = PaperDataHandler(CSV_FILE)

<ipython-input-26-c9a76506c313>:36: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S%z format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  self.df['date'] = pd.to_datetime(self.df['date'], dayfirst=True, errors='coerce')


# ---------------------------------------------
# VALIDATIONS
# ---------------------------------------------

In [ ]:
if TRADING_MODE not in ['capital', 'quantity']:
    logging.error("Invalid TRADING_MODE. Choose 'capital' or 'quantity'.")
    raise ValueError("Invalid TRADING_MODE. Choose 'capital' or 'quantity'.")

if TRADING_MODE == 'capital' and Y_CAPITAL <= 0:
    logging.error("Y_CAPITAL must be a positive number for capital-based trading.")
    raise ValueError("Invalid Y_CAPITAL for capital-based trading.")

if TRADING_MODE == 'quantity' and (not isinstance(QUANTITY, int) or QUANTITY <= 0):
    logging.error("QUANTITY must be a positive integer for quantity-based trading.")
    raise ValueError("Invalid QUANTITY for quantity-based trading.")


# ---------------------------------------------
# HELPER FUNCTIONS
# ---------------------------------------------

In [ ]:
def calculate_rsi(series, period=14):
    """
    Calculate RSI for the given price series.
    """
    delta = series.diff().dropna()
    up = delta.where(delta > 0, 0.0)
    down = -delta.where(delta < 0, 0.0)
    gain = up.rolling(window=period, min_periods=1).mean()
    loss = down.rolling(window=period, min_periods=1).mean()
    RS = gain / loss
    RSI = 100.0 - (100.0 / (1.0 + RS))
    return RSI

def get_latest_candles():
    """
    Fetch historical data for RSI calculation:
    - In paper mode, fetch from the PaperDataHandler.
    - In live mode, fetch from Zerodha (not implemented here).
    """
    if TRADING_MODE in ['capital', 'quantity']:
        df = paper_data_handler.get_historical_candles()
        return df
    else:
        # Implement live mode historical fetch if needed
        return None

def fetch_ltp():
    """
    Get the current LTP.
    - In paper mode, from PaperDataHandler.
    - In live mode, from kite.ltp.
    """
    if TRADING_MODE in ['capital', 'quantity']:
        ltp = paper_data_handler.get_current_ltp()
        return ltp
    else:
        try:
            data = kite.ltp(INSTRUMENT)
            return data[INSTRUMENT]['last_price']
        except Exception as e:
            logging.error(f"Error fetching LTP: {e}")
            return None

def place_order(transaction_type, quantity, price=None):
    """
    Simulate or place a market order.
    """
    if TRADING_MODE in ['capital', 'quantity']:
        # Paper Trading: Simulate order
        mock_order_id = f"mock_{transaction_type}_{int(time.time())}"
        logging.info(f"(Paper Trade) Order Placed: {transaction_type} Qty: {quantity}, ID: {mock_order_id}")
        return mock_order_id
    else:
        # Live Trading: Place order via Kite API
        try:
            order_id = kite.place_order(
                variety=kite.VARIETY_REGULAR,
                exchange="NSE",
                tradingsymbol=INSTRUMENT.split(':')[-1],
                transaction_type=transaction_type,
                quantity=quantity,
                order_type=kite.ORDER_TYPE_MARKET,
                product=kite.PRODUCT_MIS
            )
            logging.info(f"Order placed. Type: {transaction_type}, Qty: {quantity}, ID: {order_id}")
            return order_id
        except Exception as e:
            logging.error(f"Order placement failed: {e}")
            return None

def calculate_supertrend(df, period=10, multiplier=3):
    hl2 = (df['high'] + df['low']) / 2
    atr = (df['high'] - df['low']).rolling(window=period).mean()
    upper_band = hl2 + (multiplier * atr)
    lower_band = hl2 - (multiplier * atr)
    df['supertrend'] = np.nan
    in_uptrend = True
    for i in range(1, len(df)):
        if df['close'].iloc[i] > upper_band.iloc[i - 1]:
            in_uptrend = True
        elif df['close'].iloc[i] < lower_band.iloc[i - 1]:
            in_uptrend = False
        df.loc[i, 'supertrend'] = lower_band.iloc[i] if in_uptrend else upper_band.iloc[i]
    df['trend'] = np.where(df['close'] > df['supertrend'], 'up', 'down')
    return df

# ---------------------------------------------
# TRADE ANALYTICS
# ---------------------------------------------
# We'll store each completed trade in a list of dicts for analysis at the end.

In [ ]:
trades = []

# ---------------------------------------------
# TRADE ANALYTICS & REPORTING
# ---------------------------------------------

In [ ]:
def generate_trade_report(trades, total_realized_pnl):
    if not trades:
        logging.info("No trades were executed. No analytics available.")
        return

    # Convert to DataFrame for easier analysis
    trades_df = pd.DataFrame(trades)

    total_trades = len(trades_df)
    wins = trades_df[trades_df['pnl'] > 0].shape[0]
    losses = trades_df[trades_df['pnl'] < 0].shape[0]
    win_rate = (wins / total_trades) * 100 if total_trades > 0 else 0.0
    avg_win = trades_df[trades_df['pnl'] > 0]['pnl'].mean() if wins > 0 else 0.0
    avg_loss = trades_df[trades_df['pnl'] < 0]['pnl'].mean() if losses > 0 else 0.0

    # Calculate drawdown: track cumulative PnL after each trade and find the max drawdown
    trades_df['cumulative_pnl'] = trades_df['pnl'].cumsum()
    peak = trades_df['cumulative_pnl'].cummax()
    drawdowns = trades_df['cumulative_pnl'] - peak
    max_drawdown = drawdowns.min()

    # Total return on capital
    if TRADING_MODE == 'capital':
        return_on_capital = (total_realized_pnl / Y_CAPITAL) * 100
    else:
        # Calculate average entry price
        avg_entry_price = trades_df['entry_price'].mean()
        total_invested = QUANTITY * avg_entry_price
        return_on_capital = (total_realized_pnl / total_invested) * 100

    # Profit Factor: Gross Profit / Gross Loss
    gross_profit = trades_df[trades_df['pnl'] > 0]['pnl'].sum()
    gross_loss = abs(trades_df[trades_df['pnl'] < 0]['pnl'].sum())
    profit_factor = (gross_profit / gross_loss) if gross_loss != 0 else np.inf

    # Calculate net profit (sum of all PnL values)
    net_profit = trades_df['pnl'].sum()

    # Optional: Sharpe Ratio (requires risk-free rate and return series)
    if trades_df['pnl'].std() != 0:
        sharpe_ratio = (trades_df['pnl'].mean() / trades_df['pnl'].std()) * np.sqrt(252)  # Annualized
    else:
        sharpe_ratio = np.nan

    # Print Summary
    logging.info("----- TRADE ANALYTICS REPORT -----")
    logging.info(f"Total Trades: {total_trades}")
    logging.info(f"Wins: {wins}, Losses: {losses}")
    logging.info(f"Win Rate: {win_rate:.2f}%")
    logging.info(f"Average Win: {avg_win:.2f}, Average Loss: {avg_loss:.2f}")
    logging.info(f"Gross Profit: {gross_profit:.2f}")
    logging.info(f"Gross Loss: {gross_loss:.2f}")
    logging.info(f"Net Profit: {net_profit:.2f}")
    logging.info(f"Profit Factor: {profit_factor:.2f}")
    logging.info(f"Total Realized PnL: {total_realized_pnl:.2f}")
    logging.info(f"Return on Capital Deployed: {return_on_capital:.2f}%")
    logging.info(f"Max Drawdown: {max_drawdown:.2f}")
    logging.info(f"Sharpe Ratio: {sharpe_ratio:.2f}")

    logging.info("Per-Trade Summary:")
    logging.info(trades_df.to_string(index=False))

    save_trade_analytics_report(total_trades, wins, losses, win_rate, avg_win, avg_loss, gross_profit, gross_loss, net_profit, profit_factor, total_realized_pnl, return_on_capital, max_drawdown, sharpe_ratio)

    # Save to CSV or Excel if desired
    if REPORT_FILE:
        try:
            trades_df.to_csv(REPORT_FILE, index=False)
            logging.info(f"Trade report saved to {REPORT_FILE}")
        except Exception as e:
            logging.error(f"Failed to save trade report: {e}")

    logging.info("----- END OF REPORT -----")


# Function to save trade analytics report
def save_trade_analytics_report(total_trades, wins, losses, win_rate, avg_win, avg_loss, gross_profit, gross_loss, net_profit, profit_factor, total_realized_pnl, return_on_capital, max_drawdown, sharpe_ratio):
    report_content = (
        "----- TRADE ANALYTICS REPORT -----\n"
        f"Total Trades: {total_trades}\n"
        f"Wins: {wins}, Losses: {losses}\n"
        f"Win Rate: {win_rate:.2f}%\n"
        f"Average Win: {avg_win:.2f}, Average Loss: {avg_loss:.2f}\n"
        f"Gross Profit: {gross_profit:.2f}\n"
        f"Gross Loss: {gross_loss:.2f}\n"
        f"Net Profit: {net_profit:.2f}\n"
        f"Profit Factor: {profit_factor:.2f}\n"
        f"Total Realized PnL: {total_realized_pnl:.2f}\n"
        f"Return on Capital Deployed: {return_on_capital:.2f}%\n"
        f"Max Drawdown: {max_drawdown:.2f}\n"
        f"Sharpe Ratio: {sharpe_ratio:.2f}\n"
    )
    try:
        with open(REPORT_FILE_TXT, 'w') as report_file:
            report_file.write(report_content)
        logging.info(f"Trade analytics report saved to {REPORT_FILE_TXT}.")
    except Exception as e:
        logging.error(f"Error saving trade analytics report: {e}")

# ---------------------------------------------
# MAIN LOGIC
# ---------------------------------------------

In [ ]:
def main_loop():
    position_active = False
    buy_price = None
    last_order_id = None
    quantity = None
    total_realized_pnl = 0.0

    if TRADING_MODE in ['capital', 'quantity']:
        if paper_data_handler.df.empty:
            logging.warning("No paper trading data available. Exiting...")
            return

    while True:
        try:
            # Advance time at the start of the loop (for each iteration in paper mode)
            if TRADING_MODE in ['capital', 'quantity']:
                if paper_data_handler.current_index is None or paper_data_handler.current_index >= len(paper_data_handler.df):
                    logging.info("No more data to simulate. Exiting gracefully...")
                    break

            df = get_latest_candles()
            if df is None or df.empty:
                logging.warning("No candle data available. Will try next iteration...")
                if TRADING_MODE in ['capital', 'quantity']:
                    if not paper_data_handler.advance_time():
                        logging.info("No more data available, exiting...")
                        break
                time.sleep(CHECK_INTERVAL)
                continue

            close_prices = df['close']
            if len(close_prices) < RSI_PERIOD:
                logging.info("Not enough data to compute RSI, waiting for more candles...")
                if TRADING_MODE in ['capital', 'quantity']:
                    if not paper_data_handler.advance_time():
                        logging.info("Data ended while waiting for RSI calculation. Exiting...")
                        break
                time.sleep(CHECK_INTERVAL)
                continue

            rsi_series = calculate_rsi(close_prices, period=RSI_PERIOD)
            if rsi_series.empty:
                logging.warning("RSI series empty. Will try next iteration...")
                if TRADING_MODE in ['capital', 'quantity']:
                    if not paper_data_handler.advance_time():
                        logging.info("No more data available, exiting...")
                        break
                time.sleep(CHECK_INTERVAL)
                continue

            current_rsi = rsi_series.iloc[-1]

            # Fetch current LTP
            ltp = fetch_ltp()
            if ltp is None:
                logging.warning("Failed to fetch LTP. Will retry next iteration...")
                if TRADING_MODE in ['capital', 'quantity']:
                    if not paper_data_handler.advance_time():
                        logging.info("No more data available, exiting...")
                        break
                time.sleep(CHECK_INTERVAL)
                continue

            # Determine quantity based on trading mode
            if quantity is None:
                if TRADING_MODE == 'capital':
                    quantity = math.floor(Y_CAPITAL / ltp)
                    if quantity <= 0:
                        logging.error("Capital not sufficient to buy even 1 share.")
                        break
                elif TRADING_MODE == 'quantity':
                    quantity = QUANTITY
                    if quantity <= 0:
                        logging.error("QUANTITY must be a positive integer.")
                        break

            logging.info(f"LTP: {ltp}, RSI: {current_rsi}, Position: {position_active}, Buy_Price: {buy_price}, Quantity: {quantity}")

            # Entry Condition
            if not position_active:
                if current_rsi <= RSI_BUY_THRESHOLD:
                    order_id = place_order("BUY", quantity)
                    if order_id:
                        position_active = True
                        buy_price = ltp
                        last_order_id = order_id
                        logging.info(f"Bought at {buy_price}, Qty: {quantity}")

            else:
                # Position active - check exit conditions
                profit_per_share = ltp - buy_price
                if profit_per_share >= X_PROFIT or current_rsi >= RSI_SELL_THRESHOLD or profit_per_share <= -STOP_LOSS:
                    order_id = place_order("SELL", quantity)
                    if order_id:
                        position_active = False
                        realized_pnl = profit_per_share * quantity
                        total_realized_pnl += realized_pnl
                        logging.info(f"Sold at {ltp}, Profit/Share: {profit_per_share}, Total Realized PnL: {total_realized_pnl}")

                        # Record the trade for analytics
                        trades.append({
                            'entry_price': buy_price,
                            'exit_price': ltp,
                            'profit_per_share': profit_per_share,
                            'quantity': quantity,
                            'pnl': realized_pnl,
                            'time': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                        })

                        buy_price = None
                        last_order_id = None

            # Advance time in paper trading mode
            if TRADING_MODE in ['capital', 'quantity']:
                if not paper_data_handler.advance_time():
                    logging.info("No more data to simulate. Exiting gracefully...")
                    break

            time.sleep(CHECK_INTERVAL)

        except KeyboardInterrupt:
            logging.info("Exiting gracefully due to keyboard interrupt.")
            break
        except Exception as e:
            logging.error(f"Error in main loop: {e}")
            time.sleep(CHECK_INTERVAL)

    logging.info(f"Exiting strategy. Total Realized PnL: {total_realized_pnl}")
    generate_trade_report(trades, total_realized_pnl)

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

if __name__ == "__main__":
    main_loop()

In [ ]:
import yfinance as yf
import pandas as pd
import datetime

def fetch_stock_data_in_chunks(symbol, start_date, end_date, interval="1m", output_csv="stock_data.csv"):
    """
    Fetches historical stock data in chunks to handle limitations of Yahoo Finance API for '1m' interval.
    Args:
        symbol (str): Stock symbol (e.g., "AAPL").
        start_date (str): Start date in "YYYY-MM-DD" format.
        end_date (str): End date in "YYYY-MM-DD" format.
        interval (str): Data interval (e.g., "1m", "5m").
        output_csv (str): Output CSV file name.
    """
    try:
        start = datetime.datetime.strptime(start_date, "%Y-%m-%d")
        end = datetime.datetime.strptime(end_date, "%Y-%m-%d")

        all_data = []

        logging.info(f"Fetching {interval} data for {symbol} from {start_date} to {end_date}...")

        while start < end:
            chunk_end = start + datetime.timedelta(days=7)  # Yahoo allows up to 7 days for '1m' interval
            if chunk_end > end:
                chunk_end = end

            logging.info(f"Fetching data for {symbol} from {start.strftime('%Y-%m-%d')} to {chunk_end.strftime('%Y-%m-%d')}...")
            data = yf.download(tickers=symbol, start=start.strftime('%Y-%m-%d'), end=chunk_end.strftime('%Y-%m-%d'), interval=interval)

            if not data.empty:
                # Rename the Datetime index to 'date' and reset index
                data = data.rename_axis('date').reset_index()

                # Rename columns to match desired output
                rename_map = {
                    'date': 'date',
                    'High': 'high',
                    'Close': 'close',
                    'Low': 'low',
                    'Open': 'open',
                    'Volume': 'volume'
                }
                # Map only the desired columns
                data = data[['date', 'High', 'Close', 'Low', 'Open', 'Volume']].rename(columns=rename_map)

                all_data.append(data)
                logging.info(f"Data fetched for {symbol} from {start.strftime('%Y-%m-%d')} to {chunk_end.strftime('%Y-%m-%d')} ({len(data)} rows).")
            else:
                logging.warning(f"No data found for {symbol} from {start.strftime('%Y-%m-%d')} to {chunk_end.strftime('%Y-%m-%d')}.")

            # Move start to the next day after the current chunk
            start = chunk_end

        # Combine all chunks into one DataFrame
        if all_data:
            combined_data = pd.concat(all_data)
            combined_data.reset_index(drop=True, inplace=True)  # Ensure clean index for the final DataFrame

            # Drop the first row
            combined_data = combined_data.iloc[1:]

            # Save to CSV
            logging.info(f"Saving combined data to {output_csv}...")
            combined_data.to_csv(output_csv, index=False)
            logging.info(f"Data saved successfully to {output_csv}.")
        else:
            logging.warning("No data fetched for the entire range.")

    except Exception as e:
        logging.error(f"Error fetching stock data for {symbol}: {e}", exc_info=True)

# Example usage
# fetch_stock_data_in_chunks("SBIN.NS", "2024-01-01", "2024-12-31", interval="1m", output_csv="sbin.csv")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Load data
RSI_PERIOD = 15
SUPER_TREND_MULTIPLIER = 3
SUPER_TREND_PERIOD = 10

# Load and preprocess data
df = pd.read_csv(CSV_FILE)
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values('date').reset_index(drop=True)

# Function to calculate RSI
def calculate_rsi(series, period=14):
    delta = series.diff()
    gain = delta.where(delta > 0, 0.0)
    loss = -delta.where(delta < 0, 0.0)
    avg_gain = gain.rolling(window=period, min_periods=1).mean()
    avg_loss = loss.rolling(window=period, min_periods=1).mean()
    rs = avg_gain / avg_loss
    rsi = 100.0 - (100.0 / (1.0 + rs))
    return rsi

# Function to calculate Supertrend
def calculate_supertrend(df, period=10, multiplier=3):
    """
    Calculates the Supertrend indicator.
    Args:
        df (pd.DataFrame): DataFrame with 'high', 'low', 'close' columns.
        period (int): ATR period.
        multiplier (float): Multiplier for ATR bands.
    Returns:
        pd.DataFrame: DataFrame with 'supertrend', 'upper_band', 'lower_band', and 'trend'.
    """
    # Calculate Average True Range (ATR)
    hl2 = (df['high'] + df['low']) / 2
    df['atr'] = df['high'].combine(df['low'], max) - df['low'].combine(df['close'], min)
    df['atr'] = df['atr'].rolling(window=period).mean()

    # Calculate upper and lower bands
    df['upper_band'] = hl2 + (multiplier * df['atr'])
    df['lower_band'] = hl2 - (multiplier * df['atr'])
    df['supertrend'] = np.nan  # Initialize supertrend column

    # Initialize trend variable
    in_uptrend = True

    # Iteratively calculate Supertrend
    for i in range(1, len(df)):
        if df['close'].iloc[i] > df['upper_band'].iloc[i - 1]:
            in_uptrend = True
        elif df['close'].iloc[i] < df['lower_band'].iloc[i - 1]:
            in_uptrend = False

        # Update supertrend based on the trend direction
        if in_uptrend:
            df.loc[i, 'supertrend'] = df['lower_band'].iloc[i]
        else:
            df.loc[i, 'supertrend'] = df['upper_band'].iloc[i]

    # Identify trend direction
    df['trend'] = np.where(df['close'] > df['supertrend'], 'up', 'down')

    # Clean up intermediate columns
    df.drop(['atr'], axis=1, inplace=True)
    return df

# Calculate indicators
df['rsi'] = calculate_rsi(df['close'], RSI_PERIOD)
df = calculate_supertrend(df, period=SUPER_TREND_PERIOD, multiplier=SUPER_TREND_MULTIPLIER)

# Define Buy and Sell signals
df['buy_signal'] = np.where((df['trend'] == 'up') & (df['rsi'] < 30), df['close'], np.nan)
df['sell_signal'] = np.where((df['trend'] == 'down') & (df['rsi'] > 70), df['close'], np.nan)

# Plot results
plt.figure(figsize=(14, 10))

# Price and Supertrend Plot
plt.plot(df['date'], df['close'], label='Close Price', color='blue')
plt.plot(df['date'], df['supertrend'], label='Supertrend', color='green')

# Buy/Sell signals
plt.scatter(df['date'], df['buy_signal'], label='Buy Signal', color='green', marker='^', s=100, zorder=5)
plt.scatter(df['date'], df['sell_signal'], label='Sell Signal', color='red', marker='v', s=100, zorder=5)

plt.title('Price with Supertrend and Buy/Sell Signals')
plt.legend()
plt.grid()

# RSI Plot
plt.figure(figsize=(14, 6))
plt.plot(df['date'], df['rsi'], label='RSI', color='orange')
plt.axhline(70, color='red', linestyle='--', label='Overbought (70)')
plt.axhline(30, color='green', linestyle='--', label='Oversold (30)')

plt.scatter(df['date'], df['rsi'], c=np.where(df['rsi'] < 30, 'green', np.where(df['rsi'] > 70, 'red', 'orange')), label='RSI Signal', s=10)

plt.title('RSI with Buy/Sell Signals')
plt.legend()
plt.grid()

plt.tight_layout()
plt.show()

# Save to CSV
df.to_csv("sbin_with_signals.csv", index=False)


KeyError: 'date'